## Generation: Stuffing Documents

In [1]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [4]:
vectorstore = Chroma(
    persist_directory= './vector-store-local',
    embedding_function=OpenAIEmbeddings(model='text-embedding-ada-002')
    )

/var/folders/l0/l8bzvy196wn95cjygzkqtsfw0000gn/T/ipykernel_14961/3874744517.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [9]:
retriever = vectorstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k':3, 'lambda_multi':0.7}

)

In [5]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:

Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [6]:
chat = ChatOpenAI(
    model='gpt-4',
    model_kwargs={'seed':365},
    max_tokens = 250
)

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [7]:
question = 'What software do data scientists use?/'

In [21]:
chain = { 'context': retriever,
         'question':RunnablePassthrough() } | prompt_template | chat | StrOutputParser()

In [22]:
chain.invoke(question)

'Data scientists use various types of software, including Apache Hadoop, Apache Hbase, and Mongo DB. These software tools are specifically designed for working with big data.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'